In [1]:
import os
from dotenv import load_dotenv
import logging
import datetime as dt
from typing import Dict, Any
import requests
from openai import OpenAI

In [2]:
class PromptGenerator:
    API_KEY_ENV_NAMES = {
        "openai": "OPENAI_API_KEY",
        "weather": "OPENWEATHERMAP_API_KEY"
    }
    
    WEATHER_BASE_URL = "https://api.openweathermap.org/data/2.5/weather"
    KELVIN_TO_CELSIUS_OFFSET = 273.15
    GPT_MODEL = "gpt-4o"
    GPT_DEFAULT_TOKENS = 70
    
    
    def __init__(self):
        self.openai_api_key = self._load_env_variable(PromptGenerator.API_KEY_ENV_NAMES["openai"])
        self.weather_api_key = self._load_env_variable(PromptGenerator.API_KEY_ENV_NAMES["weather"])
        self.gpt = OpenAI(api_key=self.openai_api_key)
        
        
    @staticmethod
    def _load_env_variable(env_name: str) -> str:
        load_dotenv()
        key = os.getenv(env_name)
        if not key:
            logging.error(f"Environment variable for {env_name} is not set.")
            raise ValueError(f"Environment variable for {env_name} is not set.")
        return key
    
    
    @staticmethod
    def kelvin_to_celsius(kelvin: float) -> float:
        return kelvin - PromptGenerator.KELVIN_TO_CELSIUS_OFFSET
    
    
    @staticmethod
    def format_time(timestamp: int, timezone: int) -> str:
        return dt.datetime.fromtimestamp(timestamp + timezone).strftime('%H:%M:%S')


    def get_weather_data(self, city: str = "Milan") -> Dict[str, Any]:
        url = f"{PromptGenerator.WEATHER_BASE_URL}?q={city}&appid={self.weather_api_key}"
        try:
            response = requests.get(url)
            response.raise_for_status()
            return response.json()
        except requests.exceptions.HTTPError as e:
            logging.error(f"HTTP Error occurred while fetching weather data: {e}")
            raise
        except requests.exceptions.RequestException as e:
            logging.error(f"Request Exception occurred: {e}")
            raise
        
        
    def get_prompt(self, landmark: str = "Duomo", city: str = "Milan", verbose=True) -> str:
        try:
            weather_data = self.get_weather_data(city)
            
            timezone_offset = weather_data['timezone']
            utc_now = dt.datetime.now(dt.timezone.utc)
            local_time = utc_now + dt.timedelta(seconds=timezone_offset)
            current_time = local_time.strftime('%H:%M:%S')
            
            temp_celsius = self.kelvin_to_celsius(weather_data['main']['temp'])
            description = weather_data['weather'][0]['description']

            local_sunrise = self.format_time(weather_data['sys']['sunrise'], timezone_offset)
            local_sunset = self.format_time(weather_data['sys']['sunset'], timezone_offset)

            system_content, user_content = self.build_prompt_content(
                landmark,
                description,
                temp_celsius,
                current_time,
                local_sunrise,
                local_sunset,
            )
            
            return self.query_gpt(system_content, user_content, verbose)
        
        except Exception as e:
            logging.error(f"Error in get_prompt: {e}")
            raise
    
    
    @staticmethod
    def build_prompt_content(
            landmark: str,
            weather_description: str,
            temp_celsius: float,
            current_time,
            sunrise_time,
            sunset_time
    ):
        system_message = (
            f"You are a creative assistant specialized in generating informative prompts for image generative models "
            f"like Stable Diffusion."
        )
        
        user_message = (
            f"Write a one sentence prompt showing the most famous tourism scene in {landmark} with the weather being "
            f"{weather_description} at {temp_celsius:.1f}°C, observed at time {current_time}. Please accurately reflect the "
            f"scene within the specified time range, given that the sun rises at {sunrise_time} and sun sets at "
            f"{sunset_time}, but don't show the exact time and temperature in the prompt. Append suitable descriptive "
            f"labels you think that may appear in the scene at the end, e.g. 'animals, sheep, birds, cinematic, "
            f"scenery, 8k', etc. Here is an example of expected prompt: 'Duomo di Milano under the early morning sky "
            f"with broken clouds, the iconic cathedral subtly illuminated by streetlights and the quiet ambiance of the "
            f"city at predawn hour with a few nocturnal birds in flight. cinematic, serene, atmospheric, 8k.'"
        )
        
        return system_message, user_message
    
    
    def query_gpt(self, system_content: str, user_content: str, verbose=True) -> str:
        logging.debug(f"Asking GPT: {user_content}")
        try:
            completion = self.gpt.chat.completions.create(
                model=PromptGenerator.GPT_MODEL,
                messages=[
                    {"role": "system", "content": system_content},
                    {"role": "user", "content": user_content}
                ],
                max_tokens=PromptGenerator.GPT_DEFAULT_TOKENS
            )
            gpt_response = completion.choices[0].message.content
            
            if verbose:
                print(f"Ask GPT: {user_content}")
                print(f"GPT Response: {gpt_response}")
            
            return gpt_response
        except Exception as e:
            logging.error(f"Error occurred while generating prompt: {e}")
            raise

In [3]:
prompt_generator = PromptGenerator()
prompt_generator.get_prompt(landmark="The Great Wall", city="Beijing")

Ask GPT: Write a one sentence prompt showing the most famous tourism scene in The Great Wall with the weather being overcast clouds at 29.9°C, observed at time 22:58:26. Please accurately reflect the scene within the specified time range, given that the sun rises at 06:45:27 and sun sets at 21:43:16, but don't show the exact time and temperature in the prompt. Append suitable descriptive labels you think that may appear in the scene at the end, e.g. 'animals, sheep, birds, cinematic, scenery, 8k', etc. Here is an example of expected prompt: 'Duomo di Milano under the early morning sky with broken clouds, the iconic cathedral subtly illuminated by streetlights and the quiet ambiance of the city at predawn hour with a few nocturnal birds in flight. cinematic, serene, atmospheric, 8k.'
GPT Response: The Great Wall of China shrouded in an overcast sky, the ancient structure cloaked in the soft glow of the few remaining twilight moments, subtle lights guiding the path along the historic sto

'The Great Wall of China shrouded in an overcast sky, the ancient structure cloaked in the soft glow of the few remaining twilight moments, subtle lights guiding the path along the historic stones as night gently settles in. cinematic, majestic, atmospheric, historical, 8k.'